In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision import utils as vutils  # Import torchvision.utils
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
import torchvision
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Subset

In [2]:
def build_conv_generator(latent_dim):
    model = nn.Sequential(
        # Input: (batch_size, latent_dim, 1, 1)
        nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(True),

        # State: (batch_size, 512, 4, 4)
        nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),

        # State: (batch_size, 256, 8, 8)
        nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),

        # State: (batch_size, 128, 16, 16)
        nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),

        # State: (batch_size, 64, 32, 32)
        nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
        nn.Tanh()
        # Output: (batch_size, 3, 64, 64)
    )
    return model

In [3]:
def build_conv_discriminator():
    model = nn.Sequential(
        # Input: (batch_size, 3, 64, 64)
        nn.Conv2d(3, 64, 4, 2, 1, bias=False),
        nn.LeakyReLU(0.2, inplace=True),

        # State: (batch_size, 64, 32, 32)
        nn.Conv2d(64, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2, inplace=True),

        # State: (batch_size, 128, 16, 16)
        nn.Conv2d(128, 256, 4, 2, 1, bias=False),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2, inplace=True),

        # State: (batch_size, 256, 8, 8)
        nn.Conv2d(256, 512, 4, 2, 1, bias=False),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2, inplace=True),

        # State: (batch_size, 512, 4, 4)
        nn.Conv2d(512, 1, 4, 1, 0, bias=False),
        nn.Sigmoid()
        # Output: (batch_size, 1, 1, 1)
    )
    return model


In [4]:
def load_patternnet_dataset(root_dir, image_size=64, batch_size=64, use_half=False):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    dataset = ImageFolder(root=root_dir, transform=transform)
    
    if use_half:
        # Use only half of the dataset
        num_samples = len(dataset)
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        half_indices = indices[:num_samples // 2]
        dataset = Subset(dataset, half_indices)
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [5]:
import matplotlib.pyplot as plt

def check_data_loading(dataloader):
    # Get a single batch from the dataloader
    batch = next(iter(dataloader))
    
    # Unpack the batch (image tensors and labels)
    images, labels = batch
    
    # Print details about the batch
    print(f"Batch size: {images.size(0)}")
    print(f"Image shape: {images.size()}")
    print(f"Labels: {labels}")

    # Display a few images to visually check
    grid_img = torchvision.utils.make_grid(images[:8], nrow=4, normalize=True)
    plt.figure(figsize=(10, 5))
    plt.imshow(grid_img.permute(1, 2, 0))
    plt.title("Sample images from the dataset")
    plt.axis('off')
    plt.show()

In [6]:
def train_gan(dataloader, G, D, latent_dim, n_epochs=200, device='cpu'):
    criterion = nn.BCELoss()
    optimizer_G = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    img_shape = (3, 64, 64)
    
    # Ensure the directory exists
    output_dir = "imagesver2"
    os.makedirs(output_dir, exist_ok=True)

    for epoch in range(n_epochs):
        for i, (imgs, labels) in enumerate(dataloader):
            valid = torch.ones(imgs.size(0), 1, 1, 1).to(device)  # Adjust label shape
            fake = torch.zeros(imgs.size(0), 1, 1, 1).to(device)  # Adjust label shape

            real_imgs = imgs.to(device)
            
            # Train Generator
            optimizer_G.zero_grad()
            z = torch.randn(imgs.size(0), latent_dim).to(device)
            z = z.view(z.size(0), latent_dim, 1, 1)  # Reshape to [batch_size, latent_dim, 1, 1]
            generated_imgs = G(z)
            g_loss = criterion(D(generated_imgs), valid)
            g_loss.backward()
            optimizer_G.step()

            # Train Discriminator
            optimizer_D.zero_grad()
            real_loss = criterion(D(real_imgs), valid)
            fake_loss = criterion(D(generated_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()

            print(f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

        if epoch % 10 == 0:
            save_image(generated_imgs, f"{output_dir}/{epoch}.png", nrow=8, normalize=True)

    print("Training complete!")


In [ ]:
latent_dim = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

G = build_conv_generator(latent_dim).to(device)
D = build_conv_discriminator().to(device)

dataloader = load_patternnet_dataset(root_dir='../data')

    # Check if the data is loaded properly


train_gan(dataloader, G, D, latent_dim, n_epochs=200, device=device)

[Epoch 0/200] [Batch 0/475] [D loss: 0.7025136351585388] [G loss: 0.5983217358589172]
[Epoch 0/200] [Batch 1/475] [D loss: 0.4830281734466553] [G loss: 1.4818223714828491]
[Epoch 0/200] [Batch 2/475] [D loss: 0.47962290048599243] [G loss: 0.6313629150390625]
[Epoch 0/200] [Batch 3/475] [D loss: 0.1875494420528412] [G loss: 1.8882274627685547]
[Epoch 0/200] [Batch 4/475] [D loss: 0.3096398115158081] [G loss: 1.066463828086853]
[Epoch 0/200] [Batch 5/475] [D loss: 0.16435979306697845] [G loss: 2.0603880882263184]
[Epoch 0/200] [Batch 6/475] [D loss: 0.2691708207130432] [G loss: 1.0910660028457642]
[Epoch 0/200] [Batch 7/475] [D loss: 0.129998579621315] [G loss: 4.239904403686523]
[Epoch 0/200] [Batch 8/475] [D loss: 0.6936137676239014] [G loss: 0.33673179149627686]
[Epoch 0/200] [Batch 9/475] [D loss: 0.08317521214485168] [G loss: 8.162596702575684]
[Epoch 0/200] [Batch 10/475] [D loss: 0.08828628808259964] [G loss: 4.978071689605713]
[Epoch 0/200] [Batch 11/475] [D loss: 1.2145185470581

[Epoch 0/200] [Batch 95/475] [D loss: 0.11002247780561447] [G loss: 2.541177988052368]
[Epoch 0/200] [Batch 96/475] [D loss: 0.08484795689582825] [G loss: 2.6176984310150146]
[Epoch 0/200] [Batch 97/475] [D loss: 0.09658868610858917] [G loss: 2.565068244934082]
[Epoch 0/200] [Batch 98/475] [D loss: 0.24467700719833374] [G loss: 2.150129795074463]
[Epoch 0/200] [Batch 99/475] [D loss: 0.16607697308063507] [G loss: 1.7853095531463623]
[Epoch 0/200] [Batch 100/475] [D loss: 0.2828170359134674] [G loss: 4.379606246948242]
[Epoch 0/200] [Batch 101/475] [D loss: 0.34551212191581726] [G loss: 0.8232814073562622]
[Epoch 0/200] [Batch 102/475] [D loss: 0.1816638559103012] [G loss: 6.036231994628906]
[Epoch 0/200] [Batch 103/475] [D loss: 0.0961867943406105] [G loss: 5.658185958862305]
[Epoch 0/200] [Batch 104/475] [D loss: 0.1839998960494995] [G loss: 1.8966777324676514]
[Epoch 0/200] [Batch 105/475] [D loss: 0.37886691093444824] [G loss: 0.9837822914123535]
[Epoch 0/200] [Batch 106/475] [D los

[Epoch 0/200] [Batch 189/475] [D loss: 0.788330614566803] [G loss: 0.344606876373291]
[Epoch 0/200] [Batch 190/475] [D loss: 0.7808660268783569] [G loss: 4.590561389923096]
[Epoch 0/200] [Batch 191/475] [D loss: 0.2978839874267578] [G loss: 1.8256402015686035]
[Epoch 0/200] [Batch 192/475] [D loss: 0.3098590075969696] [G loss: 1.0232561826705933]
[Epoch 0/200] [Batch 193/475] [D loss: 0.12399286031723022] [G loss: 2.6791183948516846]
[Epoch 0/200] [Batch 194/475] [D loss: 0.20136505365371704] [G loss: 2.09480357170105]
[Epoch 0/200] [Batch 195/475] [D loss: 0.130099818110466] [G loss: 2.0873160362243652]
[Epoch 0/200] [Batch 196/475] [D loss: 0.20795898139476776] [G loss: 1.8412344455718994]
[Epoch 0/200] [Batch 197/475] [D loss: 0.19610637426376343] [G loss: 2.8986892700195312]
[Epoch 0/200] [Batch 198/475] [D loss: 0.22838401794433594] [G loss: 1.660179853439331]
[Epoch 0/200] [Batch 199/475] [D loss: 0.16324248909950256] [G loss: 2.119452476501465]
[Epoch 0/200] [Batch 200/475] [D l

[Epoch 0/200] [Batch 282/475] [D loss: 0.07530936598777771] [G loss: 3.5451605319976807]
[Epoch 0/200] [Batch 283/475] [D loss: 0.08057358860969543] [G loss: 2.766369342803955]
[Epoch 0/200] [Batch 284/475] [D loss: 0.04730165749788284] [G loss: 3.2069694995880127]
[Epoch 0/200] [Batch 285/475] [D loss: 0.09258560836315155] [G loss: 2.691953182220459]
[Epoch 0/200] [Batch 286/475] [D loss: 0.10420215129852295] [G loss: 2.583019733428955]
[Epoch 0/200] [Batch 287/475] [D loss: 0.1445232331752777] [G loss: 2.1280360221862793]
[Epoch 0/200] [Batch 288/475] [D loss: 0.12617723643779755] [G loss: 4.051766872406006]
[Epoch 0/200] [Batch 289/475] [D loss: 0.2986634075641632] [G loss: 1.648013949394226]
[Epoch 0/200] [Batch 290/475] [D loss: 0.19902008771896362] [G loss: 2.6893985271453857]
[Epoch 0/200] [Batch 291/475] [D loss: 0.2000284492969513] [G loss: 1.7987974882125854]
[Epoch 0/200] [Batch 292/475] [D loss: 0.13569383323192596] [G loss: 2.704308271408081]
[Epoch 0/200] [Batch 293/475] 

[Epoch 0/200] [Batch 375/475] [D loss: 0.17096829414367676] [G loss: 1.8501091003417969]
[Epoch 0/200] [Batch 376/475] [D loss: 0.15217316150665283] [G loss: 2.577817678451538]
[Epoch 0/200] [Batch 377/475] [D loss: 0.1453004628419876] [G loss: 2.5744850635528564]
[Epoch 0/200] [Batch 378/475] [D loss: 0.2071874737739563] [G loss: 1.8817509412765503]
[Epoch 0/200] [Batch 379/475] [D loss: 0.1347145289182663] [G loss: 2.671111822128296]
[Epoch 0/200] [Batch 380/475] [D loss: 0.19559764862060547] [G loss: 2.6057827472686768]
[Epoch 0/200] [Batch 381/475] [D loss: 0.2712889611721039] [G loss: 1.9450323581695557]
[Epoch 0/200] [Batch 382/475] [D loss: 0.33464670181274414] [G loss: 1.3555152416229248]
[Epoch 0/200] [Batch 383/475] [D loss: 0.365302175283432] [G loss: 3.257082939147949]
[Epoch 0/200] [Batch 384/475] [D loss: 0.6880055069923401] [G loss: 0.7451122403144836]
[Epoch 0/200] [Batch 385/475] [D loss: 0.6974100470542908] [G loss: 4.79668664932251]
[Epoch 0/200] [Batch 386/475] [D l

...

[Epoch 21/200] [Batch 55/475] [D loss: 0.055943869054317474] [G loss: 3.505337953567505]
[Epoch 21/200] [Batch 56/475] [D loss: 0.04271374270319939] [G loss: 4.242212295532227]
[Epoch 21/200] [Batch 57/475] [D loss: 0.03863809257745743] [G loss: 3.962735176086426]
[Epoch 21/200] [Batch 58/475] [D loss: 0.042737171053886414] [G loss: 4.2394843101501465]
[Epoch 21/200] [Batch 59/475] [D loss: 0.0159049890935421] [G loss: 4.374211311340332]
[Epoch 21/200] [Batch 60/475] [D loss: 0.06990594416856766] [G loss: 5.543006420135498]
[Epoch 21/200] [Batch 61/475] [D loss: 0.046532195061445236] [G loss: 3.5981783866882324]
[Epoch 21/200] [Batch 62/475] [D loss: 0.05532471090555191] [G loss: 3.839635133743286]
[Epoch 21/200] [Batch 63/475] [D loss: 0.060961026698350906] [G loss: 2.7523584365844727]
[Epoch 21/200] [Batch 64/475] [D loss: 0.007798298727720976] [G loss: 6.7955193519592285]
[Epoch 21/200] [Batch 65/475] [D loss: 0.017958950251340866] [G loss: 6.922399520874023]
[Epoch 21/200] [Batch 6

[Epoch 21/200] [Batch 147/475] [D loss: 0.004663147032260895] [G loss: 5.964975357055664]
[Epoch 21/200] [Batch 148/475] [D loss: 0.04839501157402992] [G loss: 4.888019561767578]
[Epoch 21/200] [Batch 149/475] [D loss: 0.03765201196074486] [G loss: 3.4503588676452637]
[Epoch 21/200] [Batch 150/475] [D loss: 0.05317281559109688] [G loss: 5.371947765350342]
[Epoch 21/200] [Batch 151/475] [D loss: 0.019264163449406624] [G loss: 4.403627872467041]
[Epoch 21/200] [Batch 152/475] [D loss: 0.012517253868281841] [G loss: 5.680521488189697]
[Epoch 21/200] [Batch 153/475] [D loss: 0.09872890263795853] [G loss: 2.581124782562256]
[Epoch 21/200] [Batch 154/475] [D loss: 0.07911328226327896] [G loss: 8.439302444458008]
[Epoch 21/200] [Batch 155/475] [D loss: 0.016177916899323463] [G loss: 6.319235801696777]
[Epoch 21/200] [Batch 156/475] [D loss: 0.011006234213709831] [G loss: 4.862867832183838]
[Epoch 21/200] [Batch 157/475] [D loss: 0.1151709109544754] [G loss: 3.544023275375366]
[Epoch 21/200] [

[Epoch 21/200] [Batch 239/475] [D loss: 0.038036245852708817] [G loss: 9.726197242736816]
[Epoch 21/200] [Batch 240/475] [D loss: 0.013389352709054947] [G loss: 5.6636643409729]
[Epoch 21/200] [Batch 241/475] [D loss: 0.05902401730418205] [G loss: 3.749276638031006]
[Epoch 21/200] [Batch 242/475] [D loss: 0.027417059987783432] [G loss: 4.357199192047119]
[Epoch 21/200] [Batch 243/475] [D loss: 0.017791779711842537] [G loss: 5.946556091308594]
[Epoch 21/200] [Batch 244/475] [D loss: 0.04756171628832817] [G loss: 5.861484050750732]
[Epoch 21/200] [Batch 245/475] [D loss: 0.00932431872934103] [G loss: 5.282342433929443]
[Epoch 21/200] [Batch 246/475] [D loss: 0.06825070828199387] [G loss: 4.958970069885254]
[Epoch 21/200] [Batch 247/475] [D loss: 0.0639084056019783] [G loss: 3.642958164215088]
[Epoch 21/200] [Batch 248/475] [D loss: 0.037541214376688004] [G loss: 3.6413393020629883]
[Epoch 21/200] [Batch 249/475] [D loss: 0.011711785569787025] [G loss: 5.2540435791015625]
[Epoch 21/200] [

[Epoch 21/200] [Batch 331/475] [D loss: 0.02519386261701584] [G loss: 6.048168182373047]
[Epoch 21/200] [Batch 332/475] [D loss: 0.05593883618712425] [G loss: 4.850375175476074]
[Epoch 21/200] [Batch 333/475] [D loss: 0.04430394247174263] [G loss: 4.304236888885498]
[Epoch 21/200] [Batch 334/475] [D loss: 0.0077941217459738255] [G loss: 6.342171669006348]
[Epoch 21/200] [Batch 335/475] [D loss: 0.033334314823150635] [G loss: 5.447899341583252]
[Epoch 21/200] [Batch 336/475] [D loss: 0.020214365795254707] [G loss: 4.04425573348999]
[Epoch 21/200] [Batch 337/475] [D loss: 0.07655786722898483] [G loss: 3.1911816596984863]
[Epoch 21/200] [Batch 338/475] [D loss: 0.23740577697753906] [G loss: 7.304338455200195]
[Epoch 21/200] [Batch 339/475] [D loss: 0.12009338289499283] [G loss: 2.4611876010894775]
[Epoch 21/200] [Batch 340/475] [D loss: 0.048154428601264954] [G loss: 7.25753927230835]
[Epoch 21/200] [Batch 341/475] [D loss: 0.018045546486973763] [G loss: 7.177433013916016]
[Epoch 21/200] 

[Epoch 21/200] [Batch 423/475] [D loss: 0.03715355694293976] [G loss: 4.591529369354248]
[Epoch 21/200] [Batch 424/475] [D loss: 0.026464855298399925] [G loss: 3.675168752670288]
[Epoch 21/200] [Batch 425/475] [D loss: 0.035585902631282806] [G loss: 4.265922546386719]
[Epoch 21/200] [Batch 426/475] [D loss: 0.06536030769348145] [G loss: 3.8290326595306396]
[Epoch 21/200] [Batch 427/475] [D loss: 0.03540252149105072] [G loss: 4.136933326721191]
[Epoch 21/200] [Batch 428/475] [D loss: 0.054553695023059845] [G loss: 2.774256944656372]
[Epoch 21/200] [Batch 429/475] [D loss: 0.05354415252804756] [G loss: 5.334591388702393]
[Epoch 21/200] [Batch 430/475] [D loss: 0.1723247915506363] [G loss: 5.80220365524292]
[Epoch 21/200] [Batch 431/475] [D loss: 0.28402408957481384] [G loss: 1.3495817184448242]
[Epoch 21/200] [Batch 432/475] [D loss: 0.05019714683294296] [G loss: 8.200298309326172]
[Epoch 21/200] [Batch 433/475] [D loss: 0.00791490264236927] [G loss: 10.226956367492676]
[Epoch 21/200] [B